In [1]:
# ✅ Step 1: Install Required Libraries
!pip install tensorflow matplotlib scikit-learn


In [3]:
# ✅ Step 2: Import Libraries
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics import classification_report, confusion_matrix

In [5]:
# ✅ Step 3: Set Dataset Path
dataset_path = r"C:\\Users\\harip\\twm\\dataset-resized"  

In [13]:
# ✅ Step 4: Load & Preprocess Dataset
img_size = (224, 224)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
print("Classes:", class_names)

Found 2527 files belonging to 6 classes.
Using 2022 files for training.
Found 2527 files belonging to 6 classes.
Using 505 files for validation.
Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [9]:
# ✅ Step 5: Prefetch for Performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [11]:
# ✅ Step 6: Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1),
])

In [17]:
# ✅ Step 7: Load MobileNetV2 base model
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

In [19]:
# ✅ Step 8: Create full model
model = tf.keras.Sequential([
    data_augmentation,
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

In [21]:
# ✅ Step 9: Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [23]:
# ✅ Step 10: Train initial model
initial_epochs = 15
history = model.fit(train_ds, validation_data=val_ds, epochs=initial_epochs)


Epoch 1/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 56s 734ms/step - accuracy: 0.4014 - loss: 1.6190 - val_accuracy: 0.5881 - val_loss: 1.1246
Epoch 2/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 43s 676ms/step - accuracy: 0.5588 - loss: 1.1466 - val_accuracy: 0.6356 - val_loss: 1.0013
Epoch 3/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 43s 679ms/step - accuracy: 0.6106 - loss: 1.0451 - val_accuracy: 0.6713 - val_loss: 0.9502
Epoch 4/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 42s 654ms/step - accuracy: 0.6186 - loss: 0.9984 - val_accuracy: 0.6812 - val_loss: 0.8890
Epoch 5/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 42s 659ms/step - accuracy: 0.6490 - loss: 0.9398 - val_accuracy: 0.6733 - val_loss: 0.9141
Epoch 6/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 42s 655ms/step - accuracy: 0.6558 - loss: 0.9392 - val_accuracy: 0.6772 - val_loss: 0.8872
Epoch 7/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 42s 655ms/step - accuracy: 0.6614 - loss: 0.8952 - val_accuracy: 0.6832 - val_loss: 0.8539
Epoch 8/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 42s 658ms/step - accuracy: 0.6662 - loss: 0.8825 - val_accu

In [25]:
# ✅ Step 11: Fine-tune model
base_model.trainable = True
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

fine_tune_epochs = 5
total_epochs = initial_epochs + fine_tune_epochs

history_fine = model.fit(train_ds,
                         validation_data=val_ds,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1])


Epoch 15/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 86s 1s/step - accuracy: 0.2985 - loss: 2.9929 - val_accuracy: 0.5762 - val_loss: 1.2642
Epoch 16/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 63s 982ms/step - accuracy: 0.4400 - loss: 1.5919 - val_accuracy: 0.5149 - val_loss: 1.5729
Epoch 17/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 62s 977ms/step - accuracy: 0.5270 - loss: 1.2753 - val_accuracy: 0.5446 - val_loss: 1.5232
Epoch 18/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 0.5687 - loss: 1.1399 - val_accuracy: 0.5762 - val_loss: 1.3493
Epoch 19/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 63s 991ms/step - accuracy: 0.5707 - loss: 1.1257 - val_accuracy: 0.6020 - val_loss: 1.2678
Epoch 20/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 62s 970ms/step - accuracy: 0.5928 - loss: 1.0585 - val_accuracy: 0.6218 - val_loss: 1.1964


In [29]:
def predict_image(image_path):
    img = tf.keras.utils.load_img(image_path, target_size=img_size)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) / 255.0
    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions)]
    confidence = np.max(predictions)
    print(f"Predicted: {predicted_class} ({confidence:.2f})")


In [31]:
# ✅ Step 14: Save the trained model
model.save("waste_classifier_model.h5")
